In [1]:
from __future__ import print_function
import os
os.chdir(os.path.dirname(os.getcwd()))

## init stereo aruco detector scene builder

In [2]:
from pkg.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

In [3]:
from pkg.planning.filtering.reach_filter import ReachTrainer
from pkg.controller.combined_robot import *

In [4]:
rtrain = ReachTrainer(scene_builder=s_builder)

## load and train

In [5]:
rtrain.load_and_learn(RobotType.indy7)

/home/rnb/.local/lib/python2.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


trainset: 	947.0 ms/1 = 946.688 ms (946.688/946.688)
testset: 	474.0 ms/1 = 473.5 ms (473.5/473.5)

trainning accuracy = 98.06 %
test accuracy = 97.76 %
trainning success accuracy = 97.77 %
trainning failure accuracy = 98.18 %
test success accuracy = 97.24 %
test failure accuracy = 97.98 %


## collect and train

In [ ]:
rtrain.collect_and_learn(RobotType.indy7, "indy0_tcp", TRAIN_COUNT=200, TEST_COUNT=100, save_data=False, save_model=False)

In [ ]:
rtrain.collect_and_learn(RobotType.panda, "panda0_hand", TRAIN_COUNT=20000, TEST_COUNT=10000, save_data=True, save_model=True)

connection_list
[False]


In [6]:
import random
from pkg.planning.scene import PlanningScene
from pkg.planning.constraint.constraint_actor import Gripper2Tool
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
self = rtrain

In [7]:
 robot_type, TIP_LINK, N_s = RobotType.panda, "panda0_hand", 100

In [8]:
self.robot_type = robot_type
# set robot
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, robot_type, None,"")], connection_list=[False])

connection_list
[False]


In [9]:

ROBOT_NAME = crob.robot_names[0]
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)

In [10]:

# create scene
gscene = self.scene_builder.create_gscene(crob, start_rviz=False)

In [11]:



# self.scene_builder.add_robot_geometries(color=(0, 1, 0, 0.5), display=True, collision=True)
# print("added robot collision boundaries")

In [12]:

pscene = PlanningScene(gscene, combined_robot=crob)

In [13]:


# make dummy binders
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TIP_LINK,
                   dims=(0.01,)*3, center=(0,0,0.0), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
pscene.create_binder(bname="grip0", gname="grip0", rname=ROBOT_NAME, _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [14]:


self.planner = MoveitPlanner(pscene)
self.planner.update_gscene()

In [15]:


gtimer = GlobalTimer.instance()
gtimer.reset()
gtimer.tic("full_loop")
featurevec_list = []
success_list = []

In [16]:
# featurevec, success, trajectory = self.sample_reaching(ROBOT_NAME, TIP_LINK, home_pose=crob.home_pose)

In [17]:
robot_name, tool_link, home_pose = ROBOT_NAME, TIP_LINK, crob.home_pose
base_link="base_link"
timeout=0.1
radius_min=0.2
radius_max=1.3
theta_min=-np.pi
theta_max=np.pi
height_min=-0.7
height_max=1.5
zenith_min=0
zenith_max=np.pi
azimuth_min=-np.pi
azimuth_max=np.pi

In [18]:
radius = random.uniform(radius_min, radius_max)
theta = random.uniform(theta_min, theta_max)
height = random.uniform(height_min, height_max)
azimuth_loc = random.uniform(azimuth_min, azimuth_max)
zenith = random.uniform(zenith_min, zenith_max)

xyz = cyl2cart(radius, theta, height)
quat = tuple(Rotation.from_dcm(hori2mat(theta, azimuth_loc, zenith)).as_quat())
goal_pose = xyz+quat

In [19]:
trajectory, success = self.planner.planner.plan_py(
    robot_name, tool_link, goal_pose, base_link, tuple(home_pose), timeout=1)

In [ ]:
gscene.urdf_content.link_map.keys()

In [ ]:
goal_pose

In [24]:
base_link

'base_link'

## load and test

In [16]:
rtrain.load_and_test(RobotType.indy7)

trainset: 	268.0 ms/1 = 268.499 ms (268.499/268.499)
testset: 	268.0 ms/1 = 267.801 ms (267.801/267.801)

trainning accuracy = 97.99 %
test accuracy = 97.01 %
trainning success accuracy = 97.15 %
trainning failure accuracy = 98.35 %
test success accuracy = 95.71 %
test failure accuracy = 97.58 %


## Disconnect stereo

In [ ]:
stereo.disconnnect()